In [5]:
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import pandas as pd
from bs4 import BeautifulSoup
import requests
import sqlite3

In [12]:
# us ski resorts

# Extract relevant information
url = "https://en.m.wikipedia.org/wiki/List_of_ski_areas_and_resorts_in_the_United_States"

# Send a GET request to the URL
response = requests.get(url)

# Check if the request was successful (status code 200)
if response.status_code == 200:
    # Parse the HTML content of the page
    soup = BeautifulSoup(response.content, 'html.parser')
    # print(soup)
    sections = soup.find_all(['h2', 'h3'])
    data = []
    str_data = []

    location_catalog = ''  
    
    for section in sections:
        if section.name == 'h2':
            headline = section.find('span', {'class': 'mw-headline'})
            if headline and ' (' in headline.text.strip():
                #print(headline.text.strip())
                location_catalog = headline.text.strip()
                location_catalog = location_catalog.split(' (')[0] 
                #print(location_catalog)
        elif section.name == 'h3' and 'mw-headline' in section.span.attrs.get('class', []):
            state = section.span.a.get('title')
            if ' (' in state:
                state = state.split(' (')[0] 
            resort_list = section.find_next('ul')
            resorts = resort_list.find_all('li')
            #print(resorts)
            for resort in resorts:
                if resort:
                    resort_name = resort.text.strip()
                    resort_name = resort_name.split(' (')[0] if ' (' in resort_name else resort_name
                    resort_name = resort_name.split(' [')[0] if '[' in resort_name else resort_name
                    resort_name = resort_name.split(' —')[0] if ' —' in resort_name else resort_name
                    #print(resort_name)
                    data.append({'location_catalog': location_catalog, 'state': state, 'resort_name': resort_name})
                    str_data.append(str(resort_name))
                
    # Create DataFrame
    ski_df = pd.DataFrame(data)
    print(ski_df)

    location_catalog        state               resort_name
0        New England  Connecticut  Mohawk Mountain Ski Area
1        New England  Connecticut         Mount Southington
2        New England  Connecticut     Powder Ridge Ski Area
3        New England  Connecticut               Ski Sundown
4        New England        Maine            Baker Mountain
..               ...          ...                       ...
461       West Coast   Washington                  Alpental
462       West Coast   Washington            Summit Central
463       West Coast   Washington               Summit East
464       West Coast   Washington               Summit West
465       West Coast   Washington                White Pass

[466 rows x 3 columns]


In [66]:
# canadian ski resorts
url = "https://en.wikipedia.org/wiki/List_of_ski_areas_and_resorts_in_Canada"

# Send a GET request to the URL
response = requests.get(url)

# Check if the request was successful (status code 200)
if response.status_code == 200:
    # Parse the HTML content of the page
    soup = BeautifulSoup(response.content, 'html.parser')
    # print(soup)
    sections = soup.find_all(['h2', 'h3'])
    data = []
    str_data = []

    location_catalog = ''  
    for section in sections:
        if section.name == 'h1':
            headline = section.find('span', {'class': 'mw-headline'})
            if headline and ' (' in headline.text.strip():
                #print(headline.text.strip())
                location_catalog = headline.text.strip()
                location_catalog = location_catalog.split(' (')[0] 
                #print(location_catalog)
        elif section.name == 'h2' and section.find('span'):
            state = section.span.text
            if state not in ['References', 'Further reading', 'External links']:
                print("state: ", state)
                if ' (' in state:
                    state = state.split(' (')[0] 
                resort_list = section.find_next('ul')
                resorts = resort_list.find_all('li')
                #print(resorts)
                for resort in resorts:
                    if resort:
                        resort_name = resort.text.strip()
                        resort_name = resort_name.split(' (')[0] if ' (' in resort_name else resort_name
                        resort_name = resort_name.split(' [')[0] if '[' in resort_name else resort_name
                        resort_name = resort_name.split(' —')[0] if ' —' in resort_name else resort_name
                        resort_name = resort_name.split(',')[0] if ',' in resort_name else resort_name
                        #print(resort_name)
                        data.append({'location_catalog': location_catalog, 'state': state, 'resort_name': resort_name})
                        str_data.append(str(resort_name))

                
    # Create DataFrame
    ski_df = pd.DataFrame(data)
    print(ski_df)

state:  Alberta
state:  British Columbia
state:  Manitoba
state:  New Brunswick
state:  Newfoundland and Labrador
state:  Northwest Territories
state:  Nova Scotia
state:  Ontario
state:  Prince Edward Island
state:  Quebec
state:  Saskatchewan
state:  Yukon
    location_catalog         state                         resort_name
0                          Alberta                      Banff Sunshine
1                          Alberta               Canmore Nordic Centre
2                          Alberta                     Canyon Ski Area
3                          Alberta              Castle Mountain Resort
4                          Alberta                       Eastlink Park
..               ...           ...                                 ...
224                   Saskatchewan         Wapiti Valley Regional Park
225                   Saskatchewan  White Butte Trails Recreation Site
226                          Yukon                      Moose Mountain
227                          Yu

In [67]:
ski_df.index = range(466, 466+len(ski_df))

In [68]:
ski_df

,location_catalog,state,resort_name
466,,Alberta,Banff Sunshine
467,,Alberta,Canmore Nordic Centre
468,,Alberta,Canyon Ski Area
469,,Alberta,Castle Mountain Resort
470,,Alberta,Eastlink Park
...,...,...,...
690,,Saskatchewan,Wapiti Valley Regional Park
691,,Saskatchewan,White Butte Trails Recreation Site
692,,Yukon,Moose Mountain
693,,Yukon,Mt. Maichen


In [69]:
for i in range(len(ski_df)):
    ski_area = ski_df.iloc[[i]]
    # print(ski_area["resort_name"].values)
    ski_resort[i+466] = {"name": f"{ski_area['resort_name'].values[0]}", "location_catalog": f"{ski_area['state'].values[0]}, CA"}

In [70]:
ski_resort

{'0': {'name': 'Mohawk Mountain Ski Area',
  'location_catalog': 'Connecticut, New England, US'},
 '1': {'name': 'Mount Southington',
  'location_catalog': 'Connecticut, New England, US'},
 '2': {'name': 'Powder Ridge Ski Area',
  'location_catalog': 'Connecticut, New England, US'},
 '3': {'name': 'Ski Sundown',
  'location_catalog': 'Connecticut, New England, US'},
 '4': {'name': 'Baker Mountain', 'location_catalog': 'Maine, New England, US'},
 '5': {'name': 'Big Rock', 'location_catalog': 'Maine, New England, US'},
 '6': {'name': 'Big Moose', 'location_catalog': 'Maine, New England, US'},
 '7': {'name': 'Black Mountain of Maine',
  'location_catalog': 'Maine, New England, US'},
 '8': {'name': 'Camden Snow Bowl',
  'location_catalog': 'Maine, New England, US'},
 '9': {'name': 'Eaton Mountain', 'location_catalog': 'Maine, New England, US'},
 '10': {'name': 'Hermon Mountain',
  'location_catalog': 'Maine, New England, US'},
 '11': {'name': 'Lonesome Pine Trails',
  'location_catalog': '

In [20]:
import sys
# Function to get location details from Google Maps API
def get_location_details(resort_name, api_key):
    print(resort_name)
    base_url = "https://maps.googleapis.com/maps/api/geocode/json"
    params = {"address": resort_name, "key": api_key}

    response = requests.get(base_url, params=params)
    data = response.json()
    #print(data)

    if data["status"] == "OK":
        result = data["results"][0]
        location = result["geometry"]["location"]
        address_components = result["address_components"]

        # Extracting relevant information
        latitude, longitude = location["lat"], location["lng"]
        address = result.get("formatted_address", "")
        state = next((component["long_name"] for component in address_components if "administrative_area_level_1" in component["types"]), "")
        city = next((component["long_name"] for component in address_components if "locality" in component["types"]), "")
        zipcode = next((component["long_name"] for component in address_components if "postal_code" in component["types"]), "")
        address = result.get("formatted_address", "")
        
        elevation_url = f'https://maps.googleapis.com/maps/api/elevation/json?locations={latitude},{longitude}&key={api_key}'
        elevation_response = requests.get(elevation_url)
        elevation_data = elevation_response.json()

        if elevation_data["status"] == "OK" and elevation_data.get("results"):
            elevation = elevation_data["results"][0].get("elevation")
        else:
            elevation = None

        #Use Places API to get additional details
        places_url = "https://maps.googleapis.com/maps/api/place/findplacefromtext/json"
        places_params = {
            "input": resort_name,
            "inputtype": "textquery",
            "fields": "place_id",
            "key": api_key
        }

        places_response = requests.get(places_url, params=places_params)
        places_data = places_response.json()

        if places_data.get("status") == "OK" and places_data.get("candidates"):
            place_id = places_data["candidates"][0]["place_id"]

            # Use Place Details API to get website
            details_url = f'https://maps.googleapis.com/maps/api/place/details/json?place_id={place_id}&key={api_key}'
            details_response = requests.get(details_url)
            details_data = details_response.json()
            url = details_data["result"].get("website", "")

            return latitude, longitude, elevation, state, city, zipcode, address, url
        else:
            return latitude, longitude, elevation, state, city, zipcode, address, None
    else:
        return None, None, None, None, None, None, None, None

# get the latitude, longtitude from google maps API and insert into databse
api_key = 'API_KEY'
err_list = []
for i, resort in ski_resorts.items():
    if int(i) < 0:
        continue
    location_catalog = resort['location_catalog']
    resort_states = location_catalog.split(',')[0]
    resort_name = resort['name']
    latitude, longitude, elevation, state, city, zipcode, address, url = get_location_details(resort_name + ', ' + resort_states, api_key)
    if zipcode:
        ski_resorts[i]['info'] = {
            "latitude": latitude,
            "longitude": longitude,
            "elevation": elevation,
            "state/province": state,
            "city": city,
            "zipcode": zipcode,
            "address": address,
            "url": url
        }
        print(ski_resorts[i])
    else:
        err_list.append(i)
        sys.stderr.write(f"Error List: {err_list}")

Mohawk Mountain Ski Area, Connecticut
{'name': 'Mohawk Mountain Ski Area', 'location_catalog': 'Connecticut, New England, US', 'info': {'latitude': 41.838231, 'longitude': -73.3150586, 'elevation': 279.6437377929688, 'state/province': 'Connecticut', 'city': 'Cornwall', 'zipcode': '06753', 'address': '46 Great Hollow Rd, Cornwall, CT 06753, USA', 'url': 'http://www.mohawkmtn.com/'}}
Mount Southington, Connecticut
{'name': 'Mount Southington', 'location_catalog': 'Connecticut, New England, US', 'info': {'latitude': 41.6064796, 'longitude': -72.8236775, 'elevation': 111.3921356201172, 'state/province': 'Connecticut', 'city': 'Southington', 'zipcode': '06489', 'address': 'E Mountain Dr, Southington, CT 06489, USA', 'url': 'https://mountsouthington.com/'}}
Powder Ridge Ski Area, Connecticut
{'name': 'Powder Ridge Ski Area', 'location_catalog': 'Connecticut, New England, US', 'info': {'latitude': 41.501645, 'longitude': -72.73645429999999, 'elevation': 114.2796249389648, 'state/province': 'C

Error List: ['6']

Black Mountain of Maine, Maine
{'name': 'Black Mountain of Maine', 'location_catalog': 'Maine, New England, US', 'info': {'latitude': 44.5769444, 'longitude': -70.6133333, 'elevation': 288.342529296875, 'state/province': 'Maine', 'city': 'Rumford', 'zipcode': '04276', 'address': '39 Glover Rd, Rumford, ME 04276, USA', 'url': 'http://www.skiblackmountain.org/'}}
Camden Snow Bowl, Maine
{'name': 'Camden Snow Bowl', 'location_catalog': 'Maine, New England, US', 'info': {'latitude': 44.2172685, 'longitude': -69.13465339999999, 'elevation': 73.49723052978516, 'state/province': 'Maine', 'city': 'Camden', 'zipcode': '04843', 'address': 'The Camden Snow Bowl, 20 Barnestown Rd, Camden, ME 04843, USA', 'url': 'http://www.camdensnowbowl.com/'}}
Eaton Mountain, Maine
{'name': 'Eaton Mountain', 'location_catalog': 'Maine, New England, US', 'info': {'latitude': 44.7681256, 'longitude': -69.6188915, 'elevation': 67.64937591552734, 'state/province': 'Maine', 'city': 'Skowhegan', 'zipcode': '04976', 'a

Error List: ['6', '15']

Pleasant Mountain, Maine
{'name': 'Pleasant Mountain', 'location_catalog': 'Maine, New England, US', 'info': {'latitude': 44.0270146, 'longitude': -70.82200879999999, 'elevation': 607.388916015625, 'state/province': 'Maine', 'city': 'Denmark', 'zipcode': '04022', 'address': 'Pleasant Mountain, Denmark, ME 04022, USA', 'url': ''}}
Powderhouse Hill, Maine
{'name': 'Powderhouse Hill', 'location_catalog': 'Maine, New England, US', 'info': {'latitude': 43.2366511, 'longitude': -70.80207349999999, 'elevation': 36.58181762695312, 'state/province': 'Maine', 'city': 'South Berwick', 'zipcode': '03908', 'address': 'Powderhouse Hill, South Berwick, ME 03908, USA', 'url': ''}}
Quoggy Jo, Maine
{'name': 'Quoggy Jo', 'location_catalog': 'Maine, New England, US', 'info': {'latitude': 46.72249980000001, 'longitude': -67.9291305, 'elevation': 226.2995758056641, 'state/province': 'Maine', 'city': 'Presque Isle', 'zipcode': '04769', 'address': '426 Fort Fairfield Rd, Presque Isle, ME 04769, USA', 'url': 

...

Error List: ['6', '15', '26', '36', '39', '52', '71', '85', '86', '97', '98', '99', '100', '101', '102', '103', '104', '106', '107', '109', '110', '112', '114', '117', '119', '121', '123', '124', '131', '139', '140', '151', '160', '163', '165', '211', '219', '222', '244', '246', '249', '251', '256', '258', '260', '263', '264', '273', '275', '328', '341', '365', '371', '373', '375', '384', '390', '394', '411', '426', '428', '433', '439', '448', '456', '462', '463', '464', '466', '468', '480', '483', '487', '490', '497', '507', '528', '534', '537', '541', '545', '552', '560', '564', '568', '569', '570', '573', '575', '577', '578', '581', '582', '588', '592', '594', '595', '599', '600', '601', '602', '604', '605', '606', '608', '610', '611', '613', '617', '634', '661', '666', '670', '672', '677', '680', '681', '682', '683', '689']

Wapiti Valley Regional Park, Saskatchewan
{'name': 'Wapiti Valley Regional Park', 'location_catalog': 'Saskatchewan, CA', 'info': {'latitude': 53.2637696, 'longitude': -104.4599622, 'elevation': 365, 'state/province': 'Saskatchewan', 'city': '', 'zipcode': 'S0E 0W0', 'address': 'Wapiti Valley Regional Park, Wapiti Noth of gronlid No. 487, Torch River No. 488, SK S0E 0W0, Canada', 'url': 'https://wapiti-valley-regional-park.business.site/?utm_source=gmb&utm_medium=referral'}}
White Butte Trails Recreation Site, Saskatchewan
{'name': 'White Butte Trails Recreation Site', 'location_catalog': 'Saskatchewan, CA', 'info': {'latitude': 50.4624396, 'longitude': -104.3792519, 'elevation': 620, 'state/province': 'Saskatchewan', 'city': 'Pilot Butte', 'zipcode': 'S0G 3Z0', 'address': 'White Butte Trails, Pilot Butte, SK S0G 3Z0, Canada', 'url': ''}}
Moose Mountain, Yukon
Mt. Maichen, Yukon


Error List: ['6', '15', '26', '36', '39', '52', '71', '85', '86', '97', '98', '99', '100', '101', '102', '103', '104', '106', '107', '109', '110', '112', '114', '117', '119', '121', '123', '124', '131', '139', '140', '151', '160', '163', '165', '211', '219', '222', '244', '246', '249', '251', '256', '258', '260', '263', '264', '273', '275', '328', '341', '365', '371', '373', '375', '384', '390', '394', '411', '426', '428', '433', '439', '448', '456', '462', '463', '464', '466', '468', '480', '483', '487', '490', '497', '507', '528', '534', '537', '541', '545', '552', '560', '564', '568', '569', '570', '573', '575', '577', '578', '581', '582', '588', '592', '594', '595', '599', '600', '601', '602', '604', '605', '606', '608', '610', '611', '613', '617', '634', '661', '666', '670', '672', '677', '680', '681', '682', '683', '689', '692']

{'name': 'Mt. Maichen', 'location_catalog': 'Yukon, CA', 'info': {'latitude': 60.1149085, 'longitude': -128.75104, 'elevation': 736.8304443359375, 'state/province': 'Yukon', 'city': 'Watson Lake', 'zipcode': 'Y0A 1C0', 'address': 'Watson Lake, YT Y0A 1C0, Canada', 'url': 'https://www.facebook.com/Watson-Lake-Ski-Club-Mt-Maichen-184564551567937/'}}
Mt Sima, Yukon


Error List: ['6', '15', '26', '36', '39', '52', '71', '85', '86', '97', '98', '99', '100', '101', '102', '103', '104', '106', '107', '109', '110', '112', '114', '117', '119', '121', '123', '124', '131', '139', '140', '151', '160', '163', '165', '211', '219', '222', '244', '246', '249', '251', '256', '258', '260', '263', '264', '273', '275', '328', '341', '365', '371', '373', '375', '384', '390', '394', '411', '426', '428', '433', '439', '448', '456', '462', '463', '464', '466', '468', '480', '483', '487', '490', '497', '507', '528', '534', '537', '541', '545', '552', '560', '564', '568', '569', '570', '573', '575', '577', '578', '581', '582', '588', '592', '594', '595', '599', '600', '601', '602', '604', '605', '606', '608', '610', '611', '613', '617', '634', '661', '666', '670', '672', '677', '680', '681', '682', '683', '689', '692', '694']

In [22]:
import json
with open('resorts.json', 'w') as fp:
    json.dump(ski_resorts, fp)

In [2]:
import json
fp = open('resorts.json', 'r')
ski_resorts = json.load(fp)

In [76]:
# do google search them feed them into llm for ratings
from bs4 import BeautifulSoup
import sys

def summarize_pages(page: str, lookfor: list):
    api_endpoint = "http://172.26.128.1:1337/v1/chat/completions"
    headers = {"Content-Type": "application/json"}
    prompt = f"here's the top webpages from a ski resort search result write short summary for {lookfor}. Begin page: {page}"
    llm_json =  {
                      "messages": [
                        {
                          "content": "Your are a helpful assistant",
                          "role": "system"
                        },
                        {
                          "content": prompt,
                          "role": "user"
                        }
                      ],
                      "model": "llama3-8b-instruct",
                      "stream": False,
                      "max_tokens": 2048,
                      "stop": [
                        "<|end_of_text|>", 
                        "<|eot_id|>"
                      ],
                      "frequency_penalty": 0,
                      "presence_penalty": 0,
                      "temperature": 0.2,
                      "top_p": 0.95
                }
    l_response = requests.post(api_endpoint, headers = headers, data=json.dumps(llm_json))
    return l_response.json()['choices'][0]['message']['content']
    
def run_llm_task(prompt: str):
    api_endpoint = "http://172.26.128.1:1337/v1/chat/completions"
    headers = {"Content-Type": "application/json"}
    llm_json =  {
                      "messages": [
                        {
                          "content": "You are a ski mountain review expert",
                          "role": "system"
                        },
                        {
                          "content": prompt,
                          "role": "user"
                        }
                      ],
                      "model": "llama3-8b-instruct",
                      "stream": False,
                      "max_tokens": 2048,
                      "stop": [
                        "<|end_of_text|>", 
                        "<|eot_id|>"
                      ],
                      "frequency_penalty": 0,
                      "presence_penalty": 0,
                      "temperature": 0.5,
                      "top_p": 0.95
                }
    l_response = requests.post(api_endpoint, headers = headers, data=json.dumps(llm_json))
    return l_response.json()['choices'][0]['message']['content']

def craw_resort_details(keyword, google_prompt: str, llm_prompt: str, f_name: str, max_retries: int = 3):
    """
    google prompt: some string that can insert ski resort name into
        eg: is %s ski resort difficult
    llm_prompt: ensure the llm prompt only output json
        eg: here's the google search result for "%s" you will rate it from 1 to 10 and return json format {resort_name: [name], %s: [score], explaination: [summary]}
    """

    with open(f_name + '.json', 'r') as fp:
        difficulty_rating = json.load(fp)
    
    with open(f_name +'_err.json', 'r') as fp:
        error_list = json.load(fp)
    
    for (i, ski_resort) in ski_resorts.items():
        if int(i) < 466:
            continue
        g_prompt = google_prompt % (ski_resort['name'] + " " + ski_resort['location_catalog'].split(',')[0])
        g_url = f"https://www.google.com/search?q={g_prompt.replace(' ', '+')}"
        print(g_url)
        user_agent='Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_1) AppleWebKit/602.2.14 (KHTML, like Gecko) Version/10.0.1 Safari/602.2.14'
        headers = {'User-Agent': user_agent,'Accept':'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8'}

        g_response = requests.get(g_url, headers=headers)
        soup = BeautifulSoup(g_response.content, 'html.parser')
        web_text = soup.get_text()

        # links = []
        # for a in soup.find_all('a', href=True):
        #     if '/url?q=' in a['href'] and 'google.com' not in a['href'] and '/search' not in a['href']:
        #         links.append(a['href'])

        # site_response = requests.get('https://www.google.com'+links[0])
        # little_soup = BeautifulSoup(site_response.content, 'html.parser')
        # cleaned_string = little_soup.get_text().encode().decode('unicode_escape').replace("\n", " ")
        # site_result = cleaned_string
        # print(summarize_pages(site_result, "skiing amenities"))
        # print('-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------')
        l_prompt = llm_prompt % (g_prompt, keyword, web_text)
        for attempt in range(max_retries):
            try:
                content = run_llm_task(l_prompt)
                
                if '}' not in content:
                    print('adding bracket -------------')
                    content += '}'

                # cut the bullshits
                content = content[content.find('{'):]
                print(i, content)
                difficulty_rating[i] = json.loads(content)
                with open(f_name + '.json', 'w') as fp:
                    json.dump(difficulty_rating, fp)
                break  # Exit the retry loop if successful
            except Exception as e:
                if attempt < max_retries - 1:
                    sys.stderr.write(f"Parse error, retrying ({attempt + 1}/{max_retries})...")
                else:
                    error_list["error"].append(i)
                    with open(f_name +'_err.json', 'w') as fp:
                        json.dump(error_list, fp)
                    sys.stderr.write(f"Error parsing: {ski_resort['name'] + ' ' + ski_resort['location_catalog'].split(',')[0]}, {e}")
                    break  # Exit the retry loop after max_retries attempts

        

In [79]:
craw_resort_details("difficulty", "is %s difficult to ski", "here's the google search result for search term '%s' you will rate it from 1 to 10 with 1 being mostly flat, 10 being very difficulty, return json format \{\"resort_name\": <name>, \"%s\": <score>, \"explaination\": <summary>\} and say nothing else, make sure the summary sounds like independent reviewer. begin search result: %s", "difficulty_report")

https://www.google.com/search?q=is+Banff+Sunshine+Alberta+difficult+to+ski
466 {
"resort_name": "Banff Sunshine Village",
"difficulty": 9,
"explaination": "Banff Sunshine Village is a challenging ski resort with a vast terrain of over 1000 meters of vertical. It offers a range of trails for all levels of skiers, from gentle green runs to extreme black and double black diamond runs. The resort's steep terrain and above-tree-line skiing can be intimidating for beginners, and even intermediate skiers may find some areas difficult to navigate. However, the resort's intuitive layout and ski patrol teams make it a safe and enjoyable experience for skiers of all levels."
}
https://www.google.com/search?q=is+Canmore+Nordic+Centre+Alberta+difficult+to+ski
467 {
"resort_name": "Canmore Nordic Centre",
"difficulty": 7,
"explaination": "While the Canmore Nordic Centre has trails for all levels, from beginners to experts, the majority of the trails are considered moderate to challenging. The trails

In [81]:
#string = "\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\nX\\n\\n\\n\\n\\n\\n\\n\\n\\nX\\n\\n\\n\\n\\n\\n\\n\\n"

# To replace actual newline characters
#cleaned_string = string.replace("\\n", " ")

# Or if you want to convert the string to an actual newline and then replace
#cleaned_string = string.encode().decode('unicode_escape').replace("\n", " ")

#print(cleaned_string)

craw_resort_details("luxury_lodging_options", "luxury %s skiing lodging", "here's the google search result for '%s' you will rate it from 1 to 10, 1 being nearly none, 10 being very good options. return one json object \{\"resort_name\": <name>, \"%s\": <score>, \"explaination\": <summary>\} and say nothing else, make sure the summary sounds like independent reviewer sound professional and avoid using words like 'I think', if there's not sufficient information give 0 score. begin search result: %s", "luxury_lodging_report")

https://www.google.com/search?q=luxury+Banff+Sunshine+Alberta+skiing+lodging
466 {
"resort_name": "Sunshine Mountain Lodge",
"luxury_lodging_options": 8,
"explaination": "Sunshine Mountain Lodge offers a range of accommodations from intimate and cozy to spectacular and sprawling suites, providing a luxurious base-camp for summer and winter adventures. The resort's on-mountain hotel offers direct access to outdoor activities, earning praise for its idyllic location and cozy ski-in/ski-out amenities. While the lodge's rooms and suites may not be as luxurious as some other high-end options in the area, its unique location and amenities make it a standout choice for skiers and snowboarders seeking a luxurious mountain experience."
}
https://www.google.com/search?q=luxury+Canmore+Nordic+Centre+Alberta+skiing+lodging
467 {
"resort_name": "Canmore Nordic Centre",
"luxury_lodging_options": 6,
"explaination": "While the search results provide a good number of options for lodging near the Canmor

Parse error, retrying (1/3)...

511 {
"resort_name": "Manning Park Resort",
"luxury_lodging_options": 2,
"explaination": "Manning Park Resort offers a range of accommodation options, including cabins and camping sites, but lacks true luxury lodgings. While the resort provides a family-friendly atmosphere and a variety of activities, the options for luxury lodging are limited. The prices listed on TripAdvisor and Agoda.com range from $98 to $250 per night, but there is no clear indication of luxury amenities or services. Overall, while Manning Park Resort is a great option for families and outdoor enthusiasts, it does not cater to those seeking luxury lodging."
}
https://www.google.com/search?q=luxury+Mount+Baldy+Ski+Area+British+Columbia+skiing+lodging
512 {
"resort_name": "Baldy Mountain Resort",
"luxury_lodging_options": 5,
"explaination": "Baldy Mountain Resort offers a range of accommodations, including the Wapiti Creek Lodge, which provides modern and rustic options directly on the ski area. Additionally, there 

Parse error, retrying (1/3)...

627 {
"resort_name": Mont Avalanche,
"luxury_lodging_options": 5,
"explaination": "While Mont Avalanche does not have a specific luxury lodging option within the resort, there are several nearby options that cater to luxury seekers. The Lodge de la Montagne and Chalets Chanteclair are two high-end options within a 10-minute drive from the resort. Additionally, the Hotel Lac Carling Golf & Spa and Cap Tremblant Mountain Resort are nearby luxury resorts that offer ski-in/ski-out access and upscale amenities. However, it's worth noting that these options may not be directly affiliated with the Mont Avalanche ski resort. Overall, while there may not be luxury lodging options within the resort itself, there are several nearby options that cater to luxury seekers."
}


Parse error, retrying (2/3)...

adding bracket -------------
627 {
"resort_name": Mont Avalanche,
"luxury_lodging_options": 6,
"explaination": "While Mont Avalanche has a limited selection of luxury lodging options, it does offer some high-end choices such as Chalet-Le Colibri and Le Grand Lodge. These options provide comfortable and spacious accommodations with modern amenities. However, the selection is limited, and prices may be higher than expected. Additionally, there are some mid-range options like Lodge de la Montagne and Chalets Chanteclair that offer a more affordable luxury experience. Overall, Mont Avalanche's luxury lodging options are decent, but not exceptional."}
https://www.google.com/search?q=luxury+Mont+Cascades+Quebec+skiing+lodging


Error parsing: Mont Avalanche Quebec, Expecting value: line 2 column 16 (char 17)

628 {
"resort_name": "Mont Cascades",
"luxury_lodging_options": 2,
"explaination": "While Mont Cascades offers some lodging options, including the North Lodge, which provides food and beverage services, changing rooms, and washrooms, there is limited information available on luxury lodging options. The resort does not seem to have any on-mountain luxury hotels or condos, and off-mountain options are not specifically marketed as luxury. However, there are some nearby hotels and B&Bs that may offer luxury amenities, such as the Wakefield Mill Hotel and Spa and the Hilton Lac-Leamy. Overall, the resort's lodging options are limited, and those seeking luxury accommodations may need to look elsewhere."
}
https://www.google.com/search?q=luxury+Mont+Castor+Quebec+skiing+lodging
629 {
"resort_name": "Mont Castor",
"luxury_lodging_options": 6,
"explaination": "Mont Castor offers a limited selection of luxury lodging options, with only a few high-end hotels and chalets listed near the resort. Wh

In [82]:
craw_resort_details("snow_quality", "How is %s snow quality skiing", "here's the google search result for '%s' you will rate it from 1 to 10, 1 for bad, 10 being very good. return one json object \{\"resort_name\": <name>, \"%s\": <score>, \"explaination\": <summary>\} and say nothing else, make sure the summary sounds like independent reviewer sound professional and avoid using words like 'I think', if there's not sufficient information give 0 score. begin search result: %s", "snow_quality")

https://www.google.com/search?q=How+is+Banff+Sunshine+Alberta+snow+quality+skiing
466 {
"resort_name": "Banff Sunshine Alberta",
"snow_quality": 8,
"explaination": "Banff Sunshine Alberta receives up to nine meters (30 feet) of snow annually, making it one of the resorts with the deepest and most reliable snowpack in Alberta. The snow stays all year, providing a consistent skiing experience. Although some reviewers mention that the snow quality is impressive, especially compared to other Banff ski resorts, others note that the resort can be liberal with opening terrain on a thin base, and the best skiing is usually in the mid and late season. Overall, the snow quality is very good, but may vary depending on the time of year and snow conditions."
}
https://www.google.com/search?q=How+is+Canmore+Nordic+Centre+Alberta+snow+quality+skiing
467 {
"resort_name": "Canmore Nordic Centre",
"snow_quality": 6,
"explaination": "The snow quality at Canmore Nordic Centre is average, with a mix of mac

Parse error, retrying (1/3)...

adding bracket -------------
500 {
"resort_name": "Fairmont Hot Springs Resort",
"snow_quality": 4,
"explaination": "Fairmont Hot Springs Resort's snow quality is decent, with varied but not extensive ski trails. The resort's snow history suggests that it can receive light rain showers, which may affect the quality of the snow. However, the resort's ski runs are great for kids and beginners, and the après-ski luxury is a plus. While there is limited information on the snow quality, the overall experience seems to be good, earning a score of 4 out of 10."}
https://www.google.com/search?q=How+is+Fernie+Alpine+Resort+British+Columbia+snow+quality+skiing
501 {
"resort_name": "Fernie Alpine Resort",
"snow_quality": 7,
"explaination": "Fernie Alpine Resort receives significant snowfall, with an average of 30 feet per year, and has a good mix of terrain for all ability levels. However, the quality of snow can vary, ranging from dry Rockies powder to soggy rain-soaked conditions. While the res

Parse error, retrying (1/3)...

adding bracket -------------
668 {
"resort_name": "Mont Sutton",
"snow_quality": 6,
"explaination": "While there is limited information available, reviews and reports suggest that Mont Sutton's snow quality can be inconsistent, with some days offering good skiing and others being icy or unskiable. The resort's snow report and webcam are not always up-to-date, making it difficult to plan a trip. However, some users report that the resort's terrain can be reliable, with some slopes remaining skiable even when the snow is poor. Overall, the snow quality at Mont Sutton appears to be average, with some good days and some poor ones."}
https://www.google.com/search?q=How+is+Station+de+neige+St-Pacome+Quebec+snow+quality+skiing
669 {
"resort_name": "Station de neige St- Pacome",
"score": 7,
"explaination": "Based on the available information, Station de neige St- Pacome has an average annual snowfall of 300 inches, which is a significant amount. However, there is limited information available 